In [1]:
box_faces = []

In [2]:
import cv2
import warnings
import numpy as np
from PIL import Image
from os import listdir
from numpy import load
from os.path import isdir
from numpy import asarray
from numpy import expand_dims
from mtcnn.mtcnn import MTCNN
from keras.models import load_model
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

Using TensorFlow backend.


In [3]:
def extract_face(filename, required_size=(160, 160)):
	image = Image.open(filename)
	image = image.convert('RGB')
	pixels = np.asarray(image)
	detector = MTCNN()
	results = detector.detect_faces(pixels)
	x1, y1, width, height = results[0]['box']
	x1, y1 = abs(x1), abs(y1)
	x2, y2 = x1 + width, y1 + height
	face = pixels[y1:y2, x1:x2]
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = np.asarray(image)
	return face_array

In [4]:
def load_faces(directory):
	faces = list()
	for filename in listdir(directory):
		path = directory + filename
		face = extract_face(path)
		faces.append(face)
	return faces

In [5]:
def load_dataset(directory):
	X, y = list(), list()
	for subdir in listdir(directory):
		path = directory + subdir + '/'
		if not isdir(path):
			continue
		faces = load_faces(path)
		labels = [subdir for _ in range(len(faces))]
		print('>loaded %d examples for class: %s' % (len(faces), subdir))
		X.extend(faces)
		y.extend(labels)
	return np.asarray(X), np.asarray(y)

In [6]:
def extract_face1(filename, required_size=(160, 160)):
	image = Image.open(filename)
	image = image.convert('RGB')
	pixels = asarray(image)
	detector = MTCNN()
	results = detector.detect_faces(pixels)
	face_array1 = []
	for a in range(len(results)):
		x1, y1, width, height = results[a]['box']
		x1, y1 = abs(x1), abs(y1)
		x2, y2 = x1 + width, y1 + height
		face = pixels[y1:y2, x1:x2]
		box_faces.append([x1, y1, width, height])
		image = Image.fromarray(face)
		image = image.resize(required_size)
		face_array1.append(asarray(image))
	return face_array1

In [7]:
def get_embedding(model, face_pixels):
	face_pixels = face_pixels.astype('float32')
	mean, std = face_pixels.mean(), face_pixels.std()
	face_pixels = (face_pixels - mean) / std
	samples = np.expand_dims(face_pixels, axis=0)
	yhat = model.predict(samples)
	return yhat[0]

In [8]:
#Create Data

In [9]:
trainX, trainy = load_dataset("C:\\Users\\Nitish_kumar\\Desktop\\nice\\data\\train\\")
print('Loaded: ', trainX.shape, trainy.shape)
model = load_model('facenet_keras.h5')
print('Loaded Model')
newTrainX = list()
for face_pixels in trainX:
	embedding = get_embedding(model, face_pixels)
	newTrainX.append(embedding)
newTrainX = np.asarray(newTrainX)
print(newTrainX.shape)
np.savez_compressed('data.npz',newTrainX, trainy)

>loaded 21 examples for class: akshay_kumar
>loaded 14 examples for class: amir_khan
>loaded 18 examples for class: ben_afflek
>loaded 21 examples for class: jerry_seinfeld
>loaded 22 examples for class: john_elton
>loaded 19 examples for class: madonna
>loaded 22 examples for class: mindy_kaling
>loaded 20 examples for class: rajini_kanth
Loaded:  (157, 160, 160, 3) (157,)


C:\Users\Nitish_kumar\Anaconda3\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Loaded Model
(157, 128)


In [10]:
data = load(r'C:\\Users\\Nitish_kumar\\Desktop\\nice\\data.npz')
trainX, trainy = data['arr_0'], data['arr_1']

in_encoder = Normalizer(norm='l2')
trainX = in_encoder.transform(trainX)

out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy = out_encoder.transform(trainy)

model = KNeighborsClassifier(n_neighbors=5)
model.fit(trainX, trainy)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [11]:
model_face = load_model(r'C:\\Users\\Nitish_kumar\\Desktop\\nice\\facenet_keras.h5')

image_path = 'data\\test\\a.jpg'

for a in extract_face1(r'C:\\Users\\Nitish_kumar\\Desktop\\nice\\'+image_path):
	test_image = get_embedding(model_face,a)
	test_image = np.array([test_image])
	test_image = in_encoder.transform(test_image)

	yhat_class = model.predict(test_image)
	yhat_prob = model.predict_proba(test_image)

	class_index = yhat_class[0]
	class_probability = yhat_prob[0,class_index] * 100
	predict_names = out_encoder.inverse_transform(yhat_class)
	print('Predicted: %s (%.3f)' % (predict_names[0], class_probability))

C:\Users\Nitish_kumar\Anaconda3\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Predicted: ben_afflek (100.000)


In [12]:
image = cv2.imread(image_path)

for a in box_faces:
	x, y, w, h = a
	cv2.rectangle(image, (x, y), (x + w, y + h), (255,0,0), 2)
	text = "%s (%.3f)" % (predict_names[0], class_probability)
	cv2.putText(image, text, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.36, (255,0,0), 2)

cv2.imshow("Image", image)
cv2.waitKey(0)

-1